# ICESat-2 Data Tutorial
---
This notebook is designed to give you a first look at ICESat-2 data (the Ice, Cloud, and land Elevation Satellite, 2). There are a range of data products generated from the raw output of the instrument, which fall into categories defined by the processing level:

* L0 -- Reconstructed, unprocessed data at full resolution (no sane person would ever look at L0 data)
* L1 -- The same reconstructed, unprocessed data, but includes ancillary information (for example, calibration parameters and georeferencing information) and is often calibrated / converted to physical units (instead of instrument voltages, which are often the fundamental measurement).
* L2 [The lowest level we would ever look at] -- Derived geophysical parameters at the same resolution as the L1 data
* L3 [The most common level to use] -- Data, derived from L2 products, that has been spatially or temporally resampled, and analyzed for additional geophysical properties.

For our purposes here, we look at the following heirarchy of products
* ATL03 [L2] -- The time, latitude, longitude, and ellipsoidal height for each photon event downlinked from ATLAS.
* ATL06 [L3] -- The geolocated estimates of land-ice surface heights and ancillary parameters that can be used to interpret the estimates and assess their quality. Photon events are aggregated into overlapping, along-track segments of a fixed length (40 m) whose centers are 20 meters apart, and the algorithm estimates the along-track slope and height at the center of each segment. 
* ATL11 [L3] -- The time series of surface heights. It is the lowest-level land-ice product that brings together data from multiple passes over the same points, obviating the need to collect individual ATL06 files for this task.
* ATL15 [L3B] -- Gridded Antarctic and Arctic Land Ice Height Change
---
To look at the data, I start by importing a series of modules that will be useful for reading and plotting the data. The ICESat-2 read functions come from Tyler Sutterly's github account (a current Postdoc at the University of Washington - https://github.com/tsutterley)

In [19]:
### sys and glob are useful tools for dealing with the local filesystem.
import sys   
import glob

### numpy and matplotlib are the core analysis packages for manipulating and plotting data arrays
import numpy as np
import matplotlib.pyplot as plt
import cmocean

### Here, we import the local data read utilities from TS
import icesat2_toolkit.utilities
from icesat2_toolkit.read_ICESat2_ATL03 import read_HDF5_ATL03
from icesat2_toolkit.read_ICESat2_ATL06 import read_HDF5_ATL06
from icesat2_toolkit.read_ICESat2_ATL11 import read_HDF5_ATL11

### Xarray is a great tool for reading NetCDF files, which are the structure for ATL14 and ATL15
import xarray

### finally, pyproj is a useful tool for converting geographic coordinates (lat/Lon) to local coordinate systems
from pyproj import Transformer
### this transformer object is designed to convert latitude/longitude to Antarctic Polarstereographic coordinates.
transformer = Transformer.from_crs("epsg:4326", "epsg:3031")

In [20]:
### In addition, we use something here called a Jupyter "Magick" -- this line changes the back-end of how Jupyter treats matplotlib plots.
### Instead of producing static images, the matplotlib widget allows you to zoom and pan and interrogate the plot interactively.
%matplotlib ipympl

# ATL03 - Global Geolocated Photon Data
---
As stated above, ATL03 is the geolocated photon product. This can be a somewhat cumbersome product to work with, but it is the basis for all of the higher level products, so it is useful to get a sense for how it is structured. Here are a few useful resources for understanding the structure of the files containing ATL03 data.

ATL03 Data Dictionary: https://nsidc.org/sites/nsidc.org/files/technical-references/ICESat2_ATL03_data_dict_v004.pdf

ATL03 File Naming Convention: ATL03_[yyyymmdd][hhmmss]\_[ttttccss]\_[vvv_rr].h5


* tttt = Reference Ground Track (RGT)
* cc = Cycle
* ss = Region
* vvv_rr = Version and revision number

---
To start working with the data, you can either find the file in the local data directory using the package `glob` (identifying any files with names that start with "ATL03") or download it directlty from the web. The cell below will first try and download the file to your local directory. If it already exists, it will instead simply read the file. 

The output of the read function is a tuple, which contains dictionaries for each file read. Because I want the `data` object to simply be a dictionary, I immediately select just the first entry in the tuple.

In [22]:
##### On Wednesdays, the NSIDC servers undergo maintenance, and so data download during that period will not work.

# On normal days, the NSIDC query requires that you have a file in your home directory called .netrc that contains
# the following info:
### machine urs.earthdata.nasa.gov login [uname] password [pwd]

try:
    # get ATL03 and ATL06 granules
    for shortname in ['ATL03','ATL06']:
        ids,urls = icesat2_toolkit.utilities.cmr(product=shortname,release='005',
            cycles=9,tracks=256,granules=10,verbose=False)
        # for each granule url from NSIDC
        for id,url in zip(ids,urls):
            icesat2_toolkit.utilities.from_nsidc(url, local=id, verbose=True)

    # get ATL11 granules
    ids,urls = icesat2_toolkit.utilities.cmr(product='ATL11',release='004',
        tracks=256,granules=10,verbose=False)

    # get ATL15 Gridded product
    ids,urls = icesat2_toolkit.utilities.cmr(product='ATL15',release='001',regions='AA',
        resolutions='01km',verbose=False)

    # for each granule url from NSIDC
    for id,url in zip(ids,urls):
        icesat2_toolkit.utilities.from_nsidc(url, local=id, verbose=True)
        
except:
    fn = glob.glob('ATL03*')
    data_03 = read_HDF5_ATL03(fn[0])[0]


Exploring dictionaries often involves a lot of the `.keys` method -- remember, dictionaries store data as "key/value pairs", so if you want to see the different storage fields, those are listed as the `keys`. The data dictionary linked above provides text-definitions of each of the keys, but you will find that the details of the data sit within first a "ground track" group (e.g., 'gt1l') and then the "heights" group.

In [ ]:
data_03['gt1l']['heights'].keys()

---
To minimize duplication in the data structure, the photon level data are grouped into segments, and their position information is all referenced to the position of a single photon within that segment. The block of code below virst identifies the index for every photons reference, and then adds the relative position to the absolute position of the reference photon to create an along-track coordinage, named `ph_x`.

In [11]:
seg_cnt = np.array(data_03['gt1l']['geolocation']['segment_ph_cnt'])
seg_cnt = np.concatenate([np.atleast_1d(0),seg_cnt])
seg_cnt = np.cumsum(seg_cnt)


ref_index = np.zeros(data_03['gt1l']['heights']['signal_conf_ph'][:,4].shape,dtype=np.int32)
for i,ind_ref in enumerate(seg_cnt[:-1]):
    ref_index[ind_ref:seg_cnt[i+1]] = i

    
ph_x = data_03['gt1l']['heights']['dist_ph_along']+data_03['gt1l']['geolocation']['segment_dist_x'][ref_index]

---
Every photon within the ATL03 product is given a "confidence classification", identifying if it is likely signal or noise. We can plot all of the photons as a function of their position along the ground track, color coded and sized by their confidence value. 

In [12]:
############ Here we define the limits to good data
start_xatc = 28500000

color_inds = np.uint16(np.linspace(0,256,12))
sizes = [0.1,0.2,0.4,0.9,1.5]
sizes = [1,1,1,1,1]
confs = [0,1,2,4]

############ And then we plot!
plt.figure(figsize=(10,5))

for i, conf_vals in enumerate(confs):
    keep_inds = np.where(np.all([[data_03['gt1l']['heights']['signal_conf_ph'][:,3] == conf_vals],[ph_x > 300000+start_xatc],[ph_x < 400000+start_xatc]],axis=0)[0])
    plt.plot(ph_x[keep_inds]-start_xatc,data_03['gt1l']['heights']['h_ph'][keep_inds],'.',ms=sizes[i],color=cmocean.cm.ice(color_inds[i]))

plt.ylabel('Elevation (m)')
plt.xlabel('Distance Along Track (m)')
#plt.xlim(300000,301000)
#plt.ylim(1420,1430)
 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Distance Along Track (m)')

Fitting of high confidence photons is then used to generate the ATL06 product, which we describe in more detail below.

# ATL06 - Land Ice Height
---
ATL06 uses a surface finding algorithm to determine the position and slope of the ice sheet surface. 

ATL06 Data Dictionary: https://nsidc.org/sites/nsidc.org/files/technical-references/ICESat2_ATL06_data_dict_v004.pdf

ATL06 File Naming Convention: ATL06_[yyyymmdd][hhmmss]\_[ttttccss]\_[vvv_rr].h5


* tttt = Reference Ground Track (RGT)
* cc = Cycle
* ss = Region
* vvv_rr = Version and revision number

---
As before, we read in the first file with a name starting "ATL06". I show here the keys of that file, which are the highest level groups in the data structure.

In [13]:
fn = glob.glob('ATL06*')
data_06 = read_HDF5_ATL06(fn[0])[0]
data_06.keys()

dict_keys(['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r', 'orbit_info', 'quality_assessment', 'ancillary_data'])

---
Most of the variabiles of interest live within the "land_ice_segments" group, so it is worth looking at those groups here.

In [14]:
data_06['gt1l']['land_ice_segments'].keys()

dict_keys(['bias_correction', 'dem', 'fit_statistics', 'geophysical', 'ground_track', 'atl06_quality_summary', 'delta_time', 'h_li', 'h_li_sigma', 'latitude', 'longitude', 'segment_id', 'sigma_geo_h'])

---
Here, we can plot the fit segments on top of the ATL03 data, to demonstrate exactly where ATL06 comes from

In [15]:

start_xatc = data_06['gt1l']['land_ice_segments']['ground_track']['x_atc'][0]

############ Here we regenerate the ATL03 plot
plt.figure()

color_inds = np.uint16(np.linspace(0,256,6))
sizes = [0.1,0.2,0.4,0.9,1.5]
sizes = [1,1,1,1,1]
confs = [0,1,2,4]
for i, conf_vals in enumerate(confs):
    keep_inds = np.where(np.all([[data_03['gt1l']['heights']['signal_conf_ph'][:,4] == conf_vals],[ph_x > 300000+start_xatc],[ph_x < 400000+start_xatc]],axis=0)[0])
    plt.plot(ph_x[keep_inds]-start_xatc,data_03['gt1l']['heights']['h_ph'][keep_inds],'.',ms=sizes[i],color=cmocean.cm.ice_r(color_inds[i]))


############ Here we define the limits to good data
keep_inds = np.where(np.all([[data_06['gt1l']['land_ice_segments']['atl06_quality_summary'] == 0],[data_06['gt1l']['land_ice_segments']['ground_track']['x_atc'] > 300000+start_xatc],[data_06['gt1l']['land_ice_segments']['ground_track']['x_atc'] < 400000+start_xatc]],axis=0)[0])

############ Here we build up the segment coordinates
at_x = np.squeeze(data_06['gt1l']['land_ice_segments']['ground_track']['x_atc'][keep_inds]-start_xatc)

at_x1 = at_x - 20;
at_x2 = at_x + 20;
at_y1 = data_06['gt1l']['land_ice_segments']['h_li'][keep_inds] - 20*data_06['gt1l']['land_ice_segments']['fit_statistics']['dh_fit_dx'][keep_inds]
at_y2 = data_06['gt1l']['land_ice_segments']['h_li'][keep_inds] + 20*data_06['gt1l']['land_ice_segments']['fit_statistics']['dh_fit_dx'][keep_inds]

############ We assemble the multidimensional array
at_x_final = np.array([[at_x1],[at_x2]]).squeeze()
at_y_final = np.array([[at_y1],[at_y2]]).squeeze()

plt.plot(at_x_final,at_y_final,color='black')

plt.ylabel('Elevation (m)')
plt.xlabel('Distance Along Track (m)')
plt.title('Ground Track 1, Left Beam')
#plt.xlim(340400,340600)
#plt.ylim(315,320)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Ground Track 1, Left Beam')

---
It can be useful to see where, on the ground, our tracks lie. Here I have the tracks plotted against the ATL14 product.

In [16]:
beams = ['gt1l','gt1r','gt2l','gt2r','gt3l','gt3r']

plt.figure()
for i,beam_opts in enumerate(beams):
    x,y = transformer.transform(data_06[beam_opts]['land_ice_segments']['latitude'],data_06[beam_opts]['land_ice_segments']['longitude'])
    plt.plot(x[keep_inds],y[keep_inds],'.',ms=1,color='black')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# ATL11 - Land-ice H(t)
---
ATL11 uses repeat observations of ATL06 to determine the height change through time. This uses each beam pair to determine the cross-track slope, to correct for height changes that are not due to changes in the glacier system but instead due to imprecision in the ground-track position.

ATL11 Data Dictionary: https://nsidc.org/sites/nsidc.org/files/technical-references/ICESat2_ATL11_data_dict_v003.pdf

ATL11 File Naming Convention: ATL11_[tttt][ss]\_[cccc]\_[vvv_rr].h5


* tttt = Reference Ground Track (RGT)
* ss = Region
* cccc = First and last cycles of data included in the file
* vvv_rr = Version and revision number

As always, we start by reading in the file, and looking at the keys of the resulting dictionary.

In [17]:
fn = glob.glob('ATL11*')
data11 = read_HDF5_ATL11(fn[0])[0]
print(data11.keys())
data11['pt1'].keys()

dict_keys(['pt1', 'pt2', 'pt3', 'orbit_info', 'quality_assessment', 'ancillary_data'])


dict_keys(['cycle_number', 'delta_time', 'h_corr', 'h_corr_sigma', 'h_corr_sigma_systematic', 'latitude', 'longitude', 'quality_summary', 'ref_pt', 'cycle_stats'])

---
Within the ATL11 data structure, there are rows in the h_corr object that signifiy the height during different cycles of data acquisition. You can see those height change results here.

In [18]:
plt.figure()

color_inds = np.uint16(np.linspace(0,256,10))

for i,cycles in enumerate(data11['pt1']['cycle_number']):
    keep_inds = np.where(np.all([[data11['pt1']['quality_summary'][:,i] == 0],[data11['pt1']['h_corr'][:,i] < 1e4]],axis=0)[0])
    plt.plot(data11['pt1']['ref_pt'][keep_inds],np.squeeze(data11['pt1']['h_corr'][keep_inds,i]),'.',color=cmocean.cm.ice_r(color_inds[i]),label='Cycle '+str(cycles))
    
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# ATL15 - Gridded Antarctic and Arctic Land Ice Height Change
---
This data set (ATL15) and ATL14 bring the time-varying height estimates provided in ATLAS/ICESat-2 L3B Annual Land Ice Height (ATL11) into a gridded format. 

ATL15 Data Dictionary: 
https://nsidc.org/sites/nsidc.org/files/technical-references/ICESat2_ATL15_data_dict_v001.pdf

ATL15 File Naming Convention: ATL15_\[RR]_\[CCCC]_\[nn]km_\[vvv_rr]


* tttt = Reference Ground Track (RGT)
* RR = RegionAntarctica = AA; Alaska = AK; Arctic Canada North = CN; Arctic Canada South = CS; Greenland and peripheral ice caps = GL; Iceland = IS; Svalbard = SV; Russian Arctic = RA.
* cccc = First and last cycles of data included in the file
* nn = Two-digit spatial resolution. Options are 01, 10, 20, and 40
* vvv_rr = Version and revision number

As always, we start by reading in the file -- this time using the package xarray (which is designed for gridded datasets)
